# 데이터 전처리

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('D:\data\키워드_통계자료_전문_조국_20190917.xlsx')[['제목', '본문내용']]

In [3]:
def isNaN(num):
    return num != num

In [4]:
senten_list = []

In [5]:
for i in df.index:
    if isNaN(df.get_value(i,'제목')):
        df.set_value(i,'제목', '')
    if isNaN(df.get_value(i,'본문내용')):
        df.set_value(i,'본문내용',  '')
              
    if df.get_value(i,'제목') + df.get_value(i,'본문내용') != '':
        senten_list.append(df.get_value(i,'제목') + df.get_value(i,'본문내용'))


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value i

In [8]:
keyword = '조국 장관'
keyword_list = keyword.split(' ')



In [16]:
senten_list = pd.Series(senten_list)

In [18]:
for i in keyword_list:
    senten_list = senten_list[list(map(lambda x: i in x, senten_list))]

In [20]:
import konlpy as kp
import numpy as np
import gensim
import re

In [21]:
senten_list = np.reshape(np.array(senten_list),(len(senten_list)))

bannedwords = ['출처','기자','하지','하기','천억','진짜','이것','저것','누구','무엇','그것','위해','역시','이런','저런','그런','무슨',
               '누가','요즘','끼리','가지','정말','보기','하나','이제','어디','라이','아주','바로','자기','그냥','지금',
               '바로','그냥','다른','이번','해주','언제','때문','완전','이건','보고','얼마나','모두','너희','우리','당신','해도','해주',
               '하라','건가','요게','그게','이게','그거','저거','지랄','대가리','이날','오전','관련','동양','입장','오후']

In [ ]:
##mecab 사전 가져오기
from konlpy.tag import Mecab
import hanja

mecab = Mecab()

##text 전체 명사화
a=[]
for i in range(len(senten_list)):
    sen = hanja.translate(senten_list[i], 'substitution')
    b=mecab.nouns(sen)
    a=a+[b]



In [ ]:
##stopword 제거
for i in range(len(a)):
    for word in list(a[i]):
        if word in bannedwords:
            a[i].remove(word)

##한글자 제거
for i in range(len(a)): 
    y=a[i]
    a[i]=[n for n in y if len(n)>1]

##list 들을 하나의 집합으로 묶기
b=[a[0]]
for i in range(1,len(a)):
    b=b+[a[i]]

In [ ]:
##bigram 생성하기
bigram = gensim.models.Phrases(b, min_count=5, threshold=50)
bigram_mod = gensim.models.phrases.Phraser(bigram)
def make_bigram(text):
    return [bigram_mod[doc] for doc in text]
words_bigram = make_bigram(b)

##문장으로 변환
bb=[' '.join(words_bigram[0])]
for i in range(1,len(words_bigram)):
    bb=bb+[' '.join(words_bigram[i])]

bb=[w.replace('인간','사람') for w in bb]
[re.sub("_", "", x) for x in bb]


# TF-IDF

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [ ]:
cv = CountVectorizer() #max_features 수정
tdm = cv.fit_transform(bb) 

##TF-IDF
tfidf = TfidfTransformer()
tdmtfidf = tfidf.fit_transform(tdm)
words=cv.get_feature_names() # 단어 추출

In [ ]:
tdmtfidf

In [ ]:

# sum tfidf frequency of each term through documents
sums = tdmtfidf.sum(axis=0)

# connecting term to its sums frequency
data = []
for col, term in enumerate(words):
    data.append( (term, sums[0,col] ))

tfidftable = pd.DataFrame(data, columns=['키워드','TF-IDF'])
tfidftable = tfidftable.sort_values('TF-IDF', ascending=False)
print('전체 글 개수 : ', len(senten_list))
print('전체 단어 개수 : ', len(words))


In [14]:
path1 = 'D:/data/' # R 작업경로

tfidftable.iloc[0:1000].to_csv(path1+'jogukTFIDF_1000.csv', encoding='ansi',index=False)  


In [15]:
path1 = 'D:/data/' # R 작업경로

tfidftable.iloc[0:100].to_csv(path1+'jogukTFIDF_100.csv', encoding='ansi',index=False)  


In [16]:
tfidftable.to_csv(path1+'jogukTFIDF_all.csv', encoding='ansi',index=False)    #전체 댓글 TFIDF

# 네트워크 매트릭스

In [17]:
tfidftdm=pd.DataFrame(tdmtfidf[0:1000].T.toarray(), index=words)

tfidftdm['TF-IDF'] = tfidftdm.sum(axis=1)


In [18]:
type(tfidftdm)

pandas.core.frame.DataFrame

In [19]:
tfidftable = pd.DataFrame(tfidftdm.sort_values('TF-IDF', ascending=False))[:1000]


In [20]:
tfidftable.shape

(1000, 1001)

In [21]:
##네트워크 매트릭스(전체)

from scipy.spatial.distance import cosine
from sklearn.metrics import pairwise_distances

tfidfmat = tfidftable.drop('TF-IDF',1)
distance_matrix=pairwise_distances(tfidfmat, metric='cosine')

In [22]:

network_matrix = pd.DataFrame(1-distance_matrix, index = tfidftable.index, columns = tfidftable.index)
netmat=network_matrix.iloc[0:1000,0:1000]
netmat.values[[np.arange(len(netmat))]*2] = 0
del netmat.index.name
del netmat.columns.name
path2 = 'C:/Users/user/Desktop/'
netmat.to_excel(path1+'joguk_netmatgephi.xlsx')

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
netmat.shape

(1000, 1000)

# LDA

In [36]:
data_sentence = bb

In [25]:
from gensim.models import Phrases
documents = ["the mayor of new york was there", "machine learning can be useful sometimes","new york mayor was present"]

sentence_stream = [doc.split(" ") for doc in documents]
bigram = Phrases(sentence_stream, min_count=1, threshold=2)
sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']
print(bigram[sent])

['the', 'mayor', 'of', 'new_york', 'was', 'there']


In [26]:
data_words = data_sentence

In [27]:
data_words[:10]

['전남 교육청 통일_희망 김미정 전라남도_교육청 운영 전남_통일 희망_열차 학교 교육청 회의실 통일_희망 열차_대장정 귀국_보고회 동안 일정 마무리 제공 전라남도_교육청 천지일보 전남_통일 희망_열차 학교 일정 귀국 천지일보 전남 김미정 유라시아_대장정 전남_통일 희망_열차 학교 교장 학생 일정 귀국_전라남도 교육청 교육감_장석웅 운영 전남_통일 희망_열차 학교 교육청 회의실 통일_희망 열차_대장정 귀국_보고회 동안 일정 마무리 귀국_보고회 장석웅_교육감 교육청 간부 직원_신민호 전라남도_의회 교육 위원회 부위원장_학부모 참석 대장정_성공 열차 학교 학생 교직원 격려 학생 지도 교원 운영 요원 구성_방단 보고회_경기도 파주_진각 중국_하얼빈 백두산_러시아 블라디보스톡_유라시아 대장정_성공 수행 과정 장석웅_교육감 학생 교직원 참석자_열정 도전 과정 기록 영상물_시청 통일_희망 열차 학교 대장정_감동 의미 공유 학생 대표 김슬기_장흥고 학생 소감 발표_백두 압록강_두만강 천지 과거 고구려_발해 통일_천지 자랑_김수민 상고 학생 압록강_방문 나라 민족 대장정 기간 동안 소감 전남_통일 희망_열차 학교 통일_희망 열차_대장정 귀국_보고회 사진 제공 전라남도_교육청 천지일보 장석웅_교육감 격려사_통일 코리아_주역 여러분_압록강 백두산_두만강 분단 현실 민족 통일 생각 여정 조국 한반도 평화통일_밑거름 신민호_전라남도 의회 교육 위원회 부위원장_대장정 고구려_유적지 백두산 민족_숨결 통일_희망 가슴 불행 역사 반복 세계 비전 시간 학교 학생 대장정 기간 동안 중국 백두산_러시아 탐방 항일 역사 통일_희망 시간 학생_반별 지도 교사_독서 토론 안중근_의사 기념관 항일_독립운동 유적지_고려인 마을 봉사_활동 자치회_활동 국제 문화 교류 다양 프로그램 진행 지난달 민족 정기 백두산_천지 통일_희망 백두산_등정 광개토_태왕 장군총_압록강 일대_고구려 유적지_탐방 대륙_고구려 기상 확인 가슴 학생 중국_하얼빈 뤼순_관동 법원_뤼순 감옥 항일 독립 운동 역사 재조명_관동 법원 

In [28]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
 
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
 
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['전', '남', ' ', '교', '육', '청', ' ', '통', '일', '_', '희', '망', ' ', '김', '미', '정', ' ', '전', '라', '남', '도', '_', '교', '육', '청', ' ', '운', '영', ' ', '전', '남', '_', '통', '일', ' ', '희', '망', '_', '열', '차', ' ', '학', '교', ' ', '교', '육', '청', ' ', '회', '의', '실', ' ', '통', '일', '_', '희', '망', ' ', '열', '차', '_', '대', '장', '정', ' ', '귀', '국', '_', '보', '고', '회', ' ', '동', '안', ' ', '일', '정', ' ', '마', '무', '리', ' ', '제', '공', ' ', '전', '라', '남', '도', '_', '교', '육', '청', ' ', '천', '지', '일', '보', ' ', '전', '남', '_', '통', '일', ' ', '희', '망', '_', '열', '차', ' ', '학', '교', ' ', '일', '정', ' ', '귀', '국', ' ', '천', '지', '일', '보', ' ', '전', '남', ' ', '김', '미', '정', ' ', '유', '라', '시', '아', '_', '대', '장', '정', ' ', '전', '남', '_', '통', '일', ' ', '희', '망', '_', '열', '차', ' ', '학', '교', ' ', '교', '장', ' ', '학', '생', ' ', '일', '정', ' ', '귀', '국', '_', '전', '라', '남', '도', ' ', '교', '육', '청', ' ', '교', '육', '감', '_', '장', '석', '웅', ' ', '운', '영', ' ', '전', '남', '_', '통', '일', ' ', '희', '망', '_', '열', '차', ' ',

In [29]:
from gensim.utils import simple_preprocess
import spacy

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))] for doc in texts]
 
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
 
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
 
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
    texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [30]:

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
 
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
 
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])
 
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
 
print(data_lemmatized[:1])

[['조국', '권력', '마무리', '조국', '법무부', '장관', '문재인', '청와대', '본관', '법무부', '뉴시스', '문재인', '대통령', '조국', '법무부', '매진_성과', '마무리', '대통령', '청와대', '의지_좌초', '생각', '이해', '대통령', '장관', '포함', '장관', '후보자', '임명', '대국민', '사과', '오늘', '장관', '임명장_수여', '국회', '장관', '임명', '찬성', '분열', '고민', '원칙', '청문회', '의혹', '임명', '강조', '취임', '대통령', '권력', '기소', '장애', '행동', '여지', '권력', '민주주의', '발전', '뉴시스']]


In [31]:
data_lemmatized

[['조국',
  '권력',
  '마무리',
  '조국',
  '법무부',
  '장관',
  '문재인',
  '청와대',
  '본관',
  '법무부',
  '뉴시스',
  '문재인',
  '대통령',
  '조국',
  '법무부',
  '매진_성과',
  '마무리',
  '대통령',
  '청와대',
  '의지_좌초',
  '생각',
  '이해',
  '대통령',
  '장관',
  '포함',
  '장관',
  '후보자',
  '임명',
  '대국민',
  '사과',
  '오늘',
  '장관',
  '임명장_수여',
  '국회',
  '장관',
  '임명',
  '찬성',
  '분열',
  '고민',
  '원칙',
  '청문회',
  '의혹',
  '임명',
  '강조',
  '취임',
  '대통령',
  '권력',
  '기소',
  '장애',
  '행동',
  '여지',
  '권력',
  '민주주의',
  '발전',
  '뉴시스']]

In [32]:
import gensim.corpora as corpora
from gensim.models import CoherenceModel

from pprint import pprint
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
 
# Create Corpus
texts = data_lemmatized
 
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
 
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 3), (4, 1), (5, 2), (6, 1), (7, 4), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 3), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 3), (25, 1), (26, 5), (27, 1), (28, 3), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 1)]]


In [33]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
id2word=id2word,
num_topics=20,
random_state=100,
update_every=1,
chunksize=100,
passes=10,
alpha='auto',
per_word_topics=True)

In [34]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.028*"장애" + 0.028*"조국" + 0.028*"의지_좌초" + 0.028*"의혹" + 0.028*"이해" + '
  '0.028*"임명" + 0.028*"임명장_수여" + 0.028*"장관" + 0.028*"오늘" + 0.028*"청와대"'),
 (1,
  '0.028*"장애" + 0.028*"조국" + 0.028*"의지_좌초" + 0.028*"의혹" + 0.028*"이해" + '
  '0.028*"임명" + 0.028*"임명장_수여" + 0.028*"장관" + 0.028*"오늘" + 0.028*"청와대"'),
 (2,
  '0.028*"장애" + 0.028*"조국" + 0.028*"의지_좌초" + 0.028*"의혹" + 0.028*"이해" + '
  '0.028*"임명" + 0.028*"임명장_수여" + 0.028*"장관" + 0.028*"오늘" + 0.028*"청와대"'),
 (3,
  '0.028*"장애" + 0.028*"조국" + 0.028*"의지_좌초" + 0.028*"의혹" + 0.028*"이해" + '
  '0.028*"임명" + 0.028*"임명장_수여" + 0.028*"장관" + 0.028*"오늘" + 0.028*"청와대"'),
 (4,
  '0.028*"장애" + 0.028*"조국" + 0.028*"의지_좌초" + 0.028*"의혹" + 0.028*"이해" + '
  '0.028*"임명" + 0.028*"임명장_수여" + 0.028*"장관" + 0.028*"오늘" + 0.028*"청와대"'),
 (5,
  '0.028*"장애" + 0.028*"조국" + 0.028*"의지_좌초" + 0.028*"의혹" + 0.028*"이해" + '
  '0.028*"임명" + 0.028*"임명장_수여" + 0.028*"장관" + 0.028*"오늘" + 0.028*"청와대"'),
 (6,
  '0.028*"장애" + 0.028*"조국" + 0.028*"의지_좌초" + 0.028*"의혹" + 0.028*"이해" + '
  '0.028*"

In [35]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -4.962973720919003


C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\scipy\sparse\lil.py:504: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not i.flags.writeable or i.dtype not in (np.int32, np.int64):
C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\scipy\sparse\lil.py:506: FutureWarning: future versions will not create a writeable array from broadcast_array. Set the writable flag explicitly to avoid this warning.
  if not j.flags.writeable or j.dtype not in (np.int32, np.int64):



Coherence Score:  0.9999999999999998


# LDA 시각화

In [36]:
# Plotting tools
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.gensim # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import pyLDAvis, pyLDAvis.sklearn
pyLDAvis.enable_notebook()


In [37]:
vect = TfidfVectorizer(lowercase=False, min_df=3)
tfidf_matrix = vect.fit_transform(data_sentence)


In [38]:
vect.get_feature_names()

['가가',
 '가감',
 '가감승제',
 '가게',
 '가격',
 '가격_매물',
 '가격_반값',
 '가격_부가가치세',
 '가격_신주',
 '가격_전세',
 '가격_폭등',
 '가격_폭락',
 '가격_품질',
 '가격대',
 '가격제',
 '가격제_거래',
 '가격제_마감',
 '가결',
 '가결_부결',
 '가결_선포',
 '가경',
 '가계',
 '가계_곤란',
 '가계_대출',
 '가계_동향',
 '가계_부채',
 '가계_소득',
 '가계_통신비',
 '가계_형편',
 '가계부',
 '가계표',
 '가고',
 '가고시마',
 '가곡',
 '가곡_마중',
 '가곡_무료',
 '가곡_선구자',
 '가공',
 '가공_파괴력',
 '가공품',
 '가관',
 '가관_가공',
 '가교',
 '가교_역할',
 '가구',
 '가구_가구',
 '가구_월평균',
 '가구_위문품',
 '가구_정전',
 '가구_주거',
 '가구_주택',
 '가구원',
 '가구주',
 '가국',
 '가권',
 '가극',
 '가금',
 '가급적',
 '가까이',
 '가까이_독방',
 '가나',
 '가나가와현',
 '가나다',
 '가나다순',
 '가나안',
 '가나자와',
 '가난',
 '가난_굴레',
 '가난_대물림',
 '가너',
 '가네',
 '가네코_후미코',
 '가늠',
 '가늠_거대',
 '가늠_시험대',
 '가늠_척도',
 '가늠자',
 '가능',
 '가다',
 '가닥',
 '가닥_초미',
 '가단_체인',
 '가담',
 '가담자',
 '가대',
 '가덕',
 '가덕_공항',
 '가도',
 '가도_게시판',
 '가독성',
 '가동',
 '가동_비상근무',
 '가동_총공세',
 '가두',
 '가두_행진',
 '가두시위',
 '가드',
 '가든',
 '가든그로브',
 '가등',
 '가등기',
 '가등기_천만',
 '가디언',
 '가라',
 '가락',
 '가락_쌀엿',
 '가락_아파트',
 '가락엿',
 '가락엿_쌀엿',
 '가란',
 '가람',
 '가랑비',
 '가래',
 '가량',

In [40]:
lda = LatentDirichletAllocation(n_components=3)
lda.fit(tfidf_matrix)
topic_term_dists = lda.components_ / lda.components_.sum(axis=1)[:, None]
doc_lengths = tfidf_matrix.sum(axis=1).getA1()
term_frequency = tfidf_matrix.sum(axis=0).getA1()
lda_doc_topic_dists = lda.transform(tfidf_matrix)
doc_topic_dists = lda_doc_topic_dists / lda_doc_topic_dists.sum(axis=1)[:, None]
vocab = vect.get_feature_names()
lda_pyldavis = pyLDAvis.prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency)
pyLDAvis.display(lda_pyldavis)

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [62]:
############시각화#############
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\user\Anaconda3\envs\tensorflow\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                                 x        y  topics  cluster       Freq
topic                                                                  
11      (-0.033070048589867035+0j)  (-0+0j)       1        1  99.172005
7      (-0.0014109376033940883+0j)  (-0+0j)       2        1   0.062486
17      (0.0004989587559693379+0j)  (-0+0j)       3        1   0.062058
19       (0.001998942790428923+0j)  (-0+0j)       4        1   0.041380
8        (0.001998942790428923+0j)  (-0+0j)       5        1   0.041380
1        (0.001998942790428923+0j)  (-0+0j)       6        1   0.041380
2        (0.001998942790428923+0j)  (-0+0j)       7        1   0.041380
3        (0.001998942790428924+0j)       0j       8        1   0.041380
4        (0.001998942790428924+0j)       0j       9        1   0.041380
5       (0.0019989427904289255+0j)       0j      10        1   0.041380
6       (0.0019989427904289255+0j)       0j      11        1   0.041380
9       (0.0019989427904289255+0j

In [ ]:
vis.token_table

# 동시등장빈도 

In [37]:
cv_vectorizer = CountVectorizer(min_df=3)
X = cv_vectorizer.fit_transform(data_sentence[:100])


In [38]:
X

<100x1163 sparse matrix of type '<class 'numpy.int64'>'
	with 9720 stored elements in Compressed Sparse Row format>

In [42]:
Xc = (X.T * X)
Xc.setdiag(0)

In [43]:
Xc = (X.T * X)

In [44]:
Xc.toarray()

array([[533,   0,   1, ...,   0,  21,   3],
       [  0,  20,   0, ...,  10,   0,   0],
       [  1,   0,  10, ...,   0,   0,   0],
       ...,
       [  0,  10,   0, ...,   5,   0,   0],
       [ 21,   0,   0, ...,   0,  14,   0],
       [  3,   0,   0, ...,   0,   0,   3]], dtype=int64)

In [44]:

Xc = (X.T * X)
Xc.setdiag(0)
#print(Xc.todense())

# pandas 를 사용하여 gephi 에서 보여줄 word-word co-occurrence matrix csv 파일을 생성
names = cv_vectorizer.get_feature_names() # 엔티티 이름들을 보여준다
df = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)
df.to_csv(path1 + 'joguk_cocur_cv_100.csv', sep = ',')

In [90]:
X[:1000].todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [1]:
data

NameError: name 'data' is not defined